In [1]:
import requests
from bs4 import BeautifulSoup
import pandas


In [2]:
base_url="http://archive.energy.gov.il/Subjects/Fuel/Pages/GxmsMniFuelStations.aspx?WPID=WPQ3&PN="

l=[]

        
for page in range(1,1000,1):
        
    if(page%10 == 0):
       print ("progress: {} pages loaded ".format (page))
      
    r=requests.get(base_url+str(page))
    c=r.content
    soup=BeautifulSoup(c,"html.parser")
    all=soup.find_all("table",{"class":"cqwpGridViewTable FillingStation"})
    
    try :

        all2=all[0].find_all("tr")[1:]

        for item in all2:
            d={}
            d["Number"]=item.find_all("td")[0].text
            d["Name"]=item.find_all("td")[1].text
            d["Company"]=item.find_all("td")[2].text
            d["Location"]=item.find_all("td")[3].text
            l.append(d)
    except:
        break
        

df=pandas.DataFrame(l)

print('Loaded')


progress: 10 pages loaded 
progress: 20 pages loaded 
progress: 30 pages loaded 
progress: 40 pages loaded 
progress: 50 pages loaded 
progress: 60 pages loaded 
progress: 70 pages loaded 
Loaded


In [3]:
#!pip install googlemaps
import googlemaps
import key as k
gmaps_key = googlemaps.Client(key = k.KEY)


df["LAT"] = None
df["LON"] = None

for i in range(len(df)):
    geocode_results = gmaps_key.geocode(df['Location'][i].replace(",", ""))
    
    try:
        lat = geocode_results[0]["geometry"]["location"]["lat"]
        lon = geocode_results[0]["geometry"]["location"]["lng"]
        df.loc[i,'LAT'] = lat
        df.loc[i,'LON'] = lon
        
    except:
        print("cannot import coordinates... next location"+ i)
        

cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot import coordinates... next location
cannot impo

In [4]:
import folium


def icon_color (company):
    if company == "טן":
        return 'lightgray'  
    elif company == "סונול":
        return 'blue' 
    elif company == "פז'":
        return 'yellow' 
    elif company == "אלון" or company == "דור":
        return 'green' 
    elif company == "דלק":
        return 'red' 
    else:
        return 'black'     

def fg(company):
    if company == "טן":
        return Ten
    elif company == "סונול":
        return Sonol
    elif company == "פז":
        return Paz
    elif company == "אלון" or company == "דור":
        return DorAlon
    elif company == "דלק":
        return Delek
    else :
        return Other



map = folium.Map(location=[32,35],zoom_start=8,tiles="Mapbox Bright")

Ten =folium.FeatureGroup(name="טן")
Sonol =folium.FeatureGroup(name="סונול")
Paz =folium.FeatureGroup(name="פז")
DorAlon =folium.FeatureGroup(name="דור אלון")
Delek =folium.FeatureGroup(name="דלק")
Other =folium.FeatureGroup(name="אחר / עצמאי")

for i in range(len(df)):
    if df["LAT"][i] == None or df["LON"][i] == None :
        break 
    else :
         gas_company = df["Company"][i]
         Marker(location=[df["LAT"][i], df["LON"][i]], popup = str(gas_company),
                                        icon=folium.Icon(color=icon_color(gas_company))).add_to(fg(gas_company))



map.add_child(Ten)
map.add_child(Sonol)
map.add_child(Paz)
map.add_child(DorAlon)
map.add_child(Delek)
map.add_child(Other)
map.add_child(folium.LayerControl())
map.save("Map.html")

In [3]:
A=df.groupby('Company',as_index=False)
A.count().sort_values(("Number"),ascending=False)

,Company,Location,Name,Number
20,פז,267,267,267
8,דלק,236,236,236
18,סונול,217,217,217
6,דור,111,111,111
3,אלון,91,91,91
2,אחר \ עצמאי,80,80,80
12,טן,51,51,51
17,"סד""ש",24,24,24
15,"מיקה תפעול תחנות דלק בע""מ",18,18,18
14,יעד,14,14,14


In [127]:
B= A.count().sort_values(("Number"),ascending=False)


In [128]:
def shortenn(str):
    
        name = str.split()
        str = name[0] +" " + name[1] + " " + name[2]
   
        return  str


for i in B.index:
    if len(B['Company'][i])>15 :
        B.at[i, 'Company'] = shortenn(B['Company'][i])
    else:
        B.at[i, 'Company'] = B['Company'][i]


In [133]:
B = B.reset_index(drop=True)


In [ ]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral11



output_file("GasStations.html")

Company = B["Company"].values
Number = B["Number"].values

C=zip(Company,Number)

cyl_cmap = factor_cmap('Company', palette= Spectral11, factors=sorted(B.Company.unique()))

source = ColumnDataSource(data=B)

p = figure(x_range=Company, plot_height=700, toolbar_location=None, title="כמות תחנות",plot_width=1500)
p.vbar(x='Company', top='Number', width=0.5, source=source, legend="Company",
        line_color='white',fill_color=cyl_cmap)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 300
p.legend.orientation = "vertical"
p.legend.location = "top_right"

show(p)

In [15]:
print ('Total Number of Gas Station : {}  !!'.format(A.count().sort_values(("Number"),ascending=False).Number.sum()))

Total Number of Gas Station : 1155  !!


In [138]:
Gas = B.iloc[:,0:2].rename(columns = {'Location' : 'NumOfGasStations'})



In [139]:
Gas.to_html('Gas.html')

